
# The Impact of Mask Wearing on COVID-19 Outbreaks in Public School

* August, 2021
* Bo Peng, Ph.D. 

## Introduction

With the increasing incidences of COVID-19 infections caused by the Delta variant, we are debating if we should enforce mask wearing in public schools.


# Assumptions

* **Community infection rate**: 
    * 2000 / 4.7 M (Harris) or 3000 / 9 M (Greater Houston Area), so about 400 / million. The true CIR should be much higher, around 1600 / million.
 
* **Size of school and vaccination coverage**: 
    * High school, size 800, assume 60% adult vaccination rate.
    * Elementary school, size 500, assume 60% vaccination rate for teachers, so ~ 3.8% overall vaccination coverage if we assume 15:1 teacher student ratio.
 
* **Transmissibility of virus**: 
    * Delta variant: r0=6.5 for symptomatic cases and 4.6 for asymptomatic carriers, incubation period 4.4, 30% asymptomatic.
 
* **Efficacy of vaccine**: 
    * Pfizer vaccine: 40% efficacy for preventing infection, around 30% efficacy for preventing secondary infection (infecting others).
 
* **Duration of simulation**:
    * 4 weeks.
 
* **Mask wearing**:
   * no mask (r0=6.5), or 
   * with mask, 80% reduction in transmission (r0=1.3 for symptomatic and 0.9 for asymptomatic).
 
* **Output**: Distribution of number of infections.
 



In [1]:
import pandas as pd

school_spec = {
    'high': {'size': 800, 'vac_coverage': 0.6 },
    'elementary': {'size': 500, 'vac_coverage': 0.6 / 15 }
}

communities = {
    'C1600': dict(community_infection_rate=1600 / 1000000),
    # 'D400': dict(community_infection_rate=400 / 1000000),
}


variants = {
    'delta': dict(sym_r0=[5, 8], asym_r0=[5*0.75, 8*0.75], incu_period=4.4, name='B.1.617.2', prop_asym_carriers=0.3),
}

# 86/76 for alpha; 76/42 for delta; and 75-81% protection against hospitalizations.
vaccination_specs = {
    'pfizer': dict(
            delta=dict(vac_immunity=[0.42, 0.38], vac_infectivity=[0.5, 0.5]),
        ),    
}

distancing_spec = {
    'no_mask': 1,
    'with_mask': 0.2
}

def get_single_context(
    school,
    distancing,
    community,
):
    variant = 'delta'
    vac_name = 'pfizer'
    return dict(
        name=f'{community}_{school}_{distancing}',
        #
        community_infection_rate=communities[community]['community_infection_rate'],
        #
        # testing
        vac_proportion=float(school_spec[school]['vac_coverage']) / 100,
        
        vac_immunity=' '.join(str(x) for x in vaccination_specs[vac_name][variant]['vac_immunity']),
        vac_infectivity=' '.join(str(x) for x in vaccination_specs[vac_name][variant]['vac_infectivity']),
                
        # constant
        pop_size=school_spec[school]['size'],
        prop_asym_carriers=variants[variant]["prop_asym_carriers"],
        
        sym_r0=f'{variants[variant]["sym_r0"][0]:.2f} {variants[variant]["sym_r0"][1]:.2f}',
        asym_r0=f'{variants[variant]["asym_r0"][0]:.2f} {variants[variant]["asym_r0"][1]:.2f}',
        incu_period=variants[variant]["incu_period"],        
        
        distancing_multiplier=distancing_spec[distancing],
        duration=28,
        #
        # use more replicates for lower community infection rate to save computing time
        num_replicates=10000)


def unique_contexts(contexts):
    names = set()
    res = []
    for context in contexts:
        if not context:
            continue
        if context['name'] in names:
            continue
        res.append(context)
        names.add(context['name'])
    return res


def get_contexts(
    school_cases=[],    
    distancing_cases=[],    
    community_cases=[],
):
    contexts = []
    for school in school_cases:
        for distancing in distancing_cases:
            for community in community_cases:
                contexts.append(
                                get_single_context(school, distancing, community))
    # remove duplicate
    return unique_contexts(contexts)


In [2]:
%preview cases -l 400

all_contexts = get_contexts(
    school_cases=['high', 'elementary'],
    community_cases=[
        'C1600',
    ],
    distancing_cases=['with_mask', 'no_mask'],
)

cases = pd.DataFrame(all_contexts)

>>> cases:

,name,community_infection_rate,vac_proportion,vac_immunity,vac_infectivity,pop_size,prop_asym_carriers,sym_r0,asym_r0,incu_period,distancing_multiplier,duration,num_replicates
0,C1600_high_with_mask,0.0016,0.0060,0.42 0.38,0.5 0.5,800,0.3,5.00 8.00,3.75 6.00,4.4,0.2,28,10000
1,C1600_high_no_mask,0.0016,0.0060,0.42 0.38,0.5 0.5,800,0.3,5.00 8.00,3.75 6.00,4.4,1.0,28,10000
2,C1600_elementary_with_mask,0.0016,0.0004,0.42 0.38,0.5 0.5,500,0.3,5.00 8.00,3.75 6.00,4.4,0.2,28,10000
3,C1600_elementary_no_mask,0.0016,0.0004,0.42 0.38,0.5 0.5,500,0.3,5.00 8.00,3.75 6.00,4.4,1.0,28,10000


In [3]:
## The following cell uses SoS workflow to submit the jobs to a cluster system, with
## output written to a temporary directory.

scratch_dir = '.'

input: for_each=all_contexts

done_if(path(f'{scratch_dir}/{name}.log').exists())

task: queue='localhost', cores=4, walltime='1h', mem='4G', tags=name, workdir=scratch_dir, trunk_size=1

sh: expand=True
  #rm -f {name}.log.lock
  outbreak_simulator --popsize {pop_size} \
      --stop-if 't>{duration}' -j 4 \
      --track-events INFECTION WARNING PLUGIN SHOW_SYMPTOM QUARANTINE \
      --repeat {num_replicates} --resume \
      --symptomatic-r0 {sym_r0} all={distancing_multiplier} \
      --asymptomatic-r0 {asym_r0} all={distancing_multiplier} \
      --incubation-period {incu_period} \
      --prop-asym-carriers {prop_asym_carriers} \
      --handle-symptomatic 'quarantine?duration=10&infected=true' \
      --logfile {name}.log \
      --plugin vaccinate \
          --start 0 --proportion {vac_proportion} --immunity {vac_immunity} \
          --infectivity {vac_infectivity} \
      --plugin community_infection --start 1 --interval 1 --probability {community_infection_rate} 

,t1ddb1f4d0d14833a,C1600_high_with_maskb043fba6b07fc533back-to-schoolcell_64e58549,Ran for 5 min 3 sec,completed


,t4b8cdb9873b4931e,C1600_high_no_maskb043fba6b07fc533back-to-schoolcell_64e58549,Ran for 25 min 22 sec,running


,t2b04d6188950cf10,C1600_elementary_with_maskb043fba6b07fc533back-to-schoolcell_64e58549,Ran for 2 min 47 sec,completed


,t16583b6df41c4001,C1600_elementary_no_maskb043fba6b07fc533back-to-schoolcell_64e58549,Ran for 27 min 38 sec,running


In [19]:
import pickle
import os
import re
import csv
import pandas as pd

def get_result(name):
    print(f'Processing {name}', flush=True)

    res = {}
    
    file = open(name + '.log', 'rU')
    reader = csv.reader(file, delimiter='\t')
    headers = next(reader, None)
    IDs = set()
    
    n_tests = 0
    n_detected = 0
    n_false_positives = 0
    n_workplace_infection = 0
    n_community_infection = 0
    n_quarantine_due_to_symptom = 0
    n_quarantine_due_to_detection = 0
    # n_replace_due_to_false_positive = 0
    n_symptoms = 0
    n_detected = 0
    n_detected_false_positive = 0
    
    for row in reader:
        IDs.add(row[0])
        if row[2] == 'PLUGIN' and 'name=testing' in row[4]:
            n_tests += int(re.match(r'.*n_tested=(\d+),.*', row[4]).group(1))
            n_detected += int(re.match(r'.*n_detected=(\d+),.*', row[4]).group(1))
            n_false_positives += int(re.match(r'.*n_false_positive=(\d+),.*', row[4]).group(1))
        elif row[2] == 'INFECTION':
            if re.match(r'by=\d', row[4]):
                n_workplace_infection += 1
            else:
                n_community_infection += 1
        elif row[2] == 'QUARANTINE':
            if 'reason=show symptom' in row[4]:
                n_quarantine_due_to_symptom += 1
            elif 'reason=detected' in row[4]:
                n_quarantine_due_to_detection += 1
            #elif '=detected,infected=False' in row[4]:
            #    n_replace_due_to_false_positive += 1
    file.close()
    
    replicates = len(IDs)
    
    # we need to scale to 2000 person weeks
    # num_events = 8 weeks * 60 people * replicates
    # num_events * 2000/replicates/8/60 = 2000 week person
    school = "high" if 'high' in name else 'elementary'
    if school == "high":
        scaling_factor = 1 / replicates
    else:
        scaling_factor = 1 / replicates

    res['Avg number of community acquired Infections'] = n_community_infection * scaling_factor
    res['Avg number of Workplace acquired Infections'] = n_workplace_infection * scaling_factor
    res['Avg number of infections'] = (
        n_community_infection + n_workplace_infection) * scaling_factor
    res['Avg of symptomatic students'] = n_quarantine_due_to_symptom * scaling_factor
    res['num_replicates'] = replicates

    with open(name + '.pickle', 'wb') as outfile:
        pickle.dump(res, outfile)
    return res

In [20]:
for name in [x['name'] for x in all_contexts]:
    get_result(name)

Processing C1600_high_with_mask
Processing C1600_high_no_mask
Processing C1600_elementary_with_mask
Processing C1600_elementary_no_mask


## Results

The following table lists some important statistics obtained from running 100,000 simulations for each scenario. The result is also available in EXCEL format. The columns of the table are:



In [21]:
## This function retrieves results for all contexts as a Python DataFrame.

import pickle
import os
import pandas as pd

def get_results(contexts):
    all_res = []
    for context in contexts:
        res_file = path(f'{context["name"]}.pickle')
        if not res_file.exists():
            print(f'Missing {context["name"]}')
            continue
        with open(res_file, 'rb') as infile:
            loaded = pickle.load(infile)
            if not 'num_replicates' in loaded or loaded["num_replicates"] < context["num_replicates"]:
                print(f'WRONG {context["name"]}')
                #os.remove(res_file)
                #continue
            res = {
                'name': context['name'],                
#                 'Vaccination coverage (%)': int(context['vac_coverage'] * 100),
                
            }
            res.update(loaded)
            res.pop('num_replicates', None)
            all_res.append(res)

    all_res = pd.DataFrame(all_res)
    return all_res

pd.options.display.float_format = '{:,.3f}'.format


In [22]:
%preview  res -l 4000

res = get_results(all_contexts)

with pd.ExcelWriter(path(f'results.xlsx'), engine='xlsxwriter') as writer: 
    res.to_excel(writer, index=False, sheet_name='All')
    
    workbook  = writer.book
    
    fmt_num = workbook.add_format({'num_format': '0.00'})
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True})
    
    worksheet = writer.sheets["All"]
    worksheet.set_column('J:P', None, fmt_num)
    #worksheet.set_row(0, 0, header_format)

WRONG C1600_high_no_mask
WRONG C1600_elementary_no_mask


>>> res:

,name,Avg number of community acquired Infections,Avg number of Workplace acquired Infections,Avg number of infections,Avg of symptomatic students
0,C1600_high_with_mask,33.957,58.155,92.112,51.471
1,C1600_high_no_mask,14.836,787.681,802.517,560.320
2,C1600_elementary_with_mask,21.264,36.862,58.126,32.494
3,C1600_elementary_no_mask,9.420,492.134,501.554,349.190


In [ ]:
res.to_csv(path('#home/BoPeng/winddown/results.csv'))

In [ ]:
%cd #home/BoPeng/winddown/
res = read.csv('results.csv', check.names=FALSE)

In [ ]:
names(res)

In [ ]:
head(res)

In [ ]:
library(ggplot2)

In [ ]:
plot_heatmap <- function(data, production, npi, variant, scale, rate) {
#     data = res
#     scale = 'All'
#     rate = 'total'
#    variant = 'delta'
#    npi = 1
    
    selected = data
    no_vac = data[data$`Vaccination efficacy at preventing transmission` == 'NOVAC', ]
    for (vac in c('VAC50', 'VAC70', 'VAC95')) {
        no_vac$`Vaccination efficacy at preventing transmission` = vac
        no_vac$`Vaccination coverage (%)` = 0
        selected = rbind(selected, no_vac)
    }
    selected = selected[selected$`Vaccination efficacy at preventing transmission` != 'NOVAC', ]
    
    selected = selected[selected$`Variant` == variant, ]
    if (rate == 'workplace') {
        title = paste('Production acquired infections,', npi * 100, '% workplace interactions', production, 'production, ', variant, 'variant')
    } else if ( rate == 'total' ) {
        title = paste('Total number of infections,', npi * 100, '% workplace interactions', production, 'production, ', variant, 'variant')
    } else if ( rate == 'symptomatic' ) {
        title = paste('Total number of symptomatic workers,', npi * 100, '% workplace interactions', production, 'production, ', variant, 'variant') 
    }
    
    if (production == 'small') {
        selected = selected[selected$`Size of production` == '60x8 weeks',]
    } else {
        selected = selected[selected$`Size of production` != '60x8 weeks',]
    }
    filename = paste0(ifelse(production=='small', 'SM', 'LG'), '_', 
                      ifelse(variant=='alpha', 'al', ifelse(variant=='delta', 'de', 'or')), 
                      '_', ifelse(scale == 'All', 'all', 'unv'), '_npi', npi, '_', rate)        
    
    selected = selected[selected$NPI == npi, ]
    
    no_test = selected[selected$`Test name` == 'NOTEST', ]
    selected = selected[selected$`Scale of testing` == scale, ]
    title = paste0(title, ', testing ', tolower(scale))
    if (scale == 'Unvaccinated') {
        selected = rbind(selected, no_test)
    }
    
    #
    pdf(paste0(filename, '.pdf'), width=10, height=8)
    
    rate_breaks = c(0, 0.1, 1, 10, 100, Inf)
    rate_labels = c("0 - 0.1", "0.1 - 1", "1 - 10", "10 - 100", "> 100")
    rate_colors = c("purple", "blue",  "orange", "brown", "red")
    selected$`CIR: Daily Infections per 100,000` = as.factor(selected$`Community infection rate (new daily cases per 100,000)`)
    selected$`Vaccination coverage (%)` = as.factor(selected$`Vaccination coverage (%)`)
    
    selected$`Test Strategy` = factor(paste(selected$`Test name`, selected$`Test frequency`),
                                      levels=c( 'PCR90 Daily', 'Ag80 Daily', 'PCR90 MWF', 'Ag80 MWF', 'PCR90 M' , 'Ag80 M', 'NOTEST ' ))
  
    if (rate == 'workplace') {
        selected$`Production Acquired Infections` = cut(selected$`Number of Workplace acquired Infections (per 2000 person-week)`, 
                                      breaks=rate_breaks, right = FALSE,
                                     labels=rate_labels)   
        fig <- ggplot(selected, aes(`CIR: Daily Infections per 100,000`,
               `Vaccination coverage (%)`, fill=`Production Acquired Infections`)) +
            scale_fill_manual(breaks=rate_labels, values=rate_colors) + 
            geom_tile() + geom_text(aes(
                label=round(`Number of Workplace acquired Infections (per 2000 person-week)`, 2)),
            colour='white', size=1.5) + 
            facet_grid(rows=vars(`Test Strategy`), 
                             cols=vars(`Vaccination efficacy at preventing transmission`)) +
            labs(title=title)
        
    } else if ( rate == 'total' ) {        
          selected$`Number of Infections` = cut(selected$`Total number of Infections (per 2000 person-week)`, 
                                      breaks=rate_breaks, right = FALSE,
                                     labels=rate_labels)
          fig <- ggplot(selected, aes(`CIR: Daily Infections per 100,000`,
               `Vaccination coverage (%)`, fill=`Number of Infections`)) +
            scale_fill_manual(breaks=rate_labels, values=rate_colors) + 
            geom_tile() + geom_text(aes(
                label=round(`Total number of Infections (per 2000 person-week)`, 2)),
                colour='white', size=1.5) + 
            facet_grid(rows=vars(`Test Strategy`), 
                             cols=vars(`Vaccination efficacy at preventing transmission`)) +
            labs(title=title)
    } else if ( rate == 'symptomatic' ) {
          selected$`Number of Symptomatic Workers` = cut(selected$`Number of Symptomatic workforce (per 2000 person-week)`, 
                                      breaks=rate_breaks, right = FALSE,
                                     labels=rate_labels)
          fig <- ggplot(selected, aes(`CIR: Daily Infections per 100,000`,
               `Vaccination coverage (%)`, fill=`Number of Symptomatic Workers`)) +
            scale_fill_manual(breaks=rate_labels, values=rate_colors) + 
            geom_tile() + geom_text(aes(
                label=round(`Total number of Infections (per 2000 person-week)`, 2)),
                colour='white', size=1.5) + 
            facet_grid(rows=vars(`Test Strategy`), 
                             cols=vars(`Vaccination efficacy at preventing transmission`)) +
            labs(title=title)
    }

    print(fig)
    dev.off()
}


In [ ]:
    
    for (production in c('small', 'large')) {
        for (npi in c(0.6, 1, 1.2)) {
            for (scale in c('All', 'Unvaccinated')) {
                for (rate in c('workplace', 'total', 'symptomatic')) {
                    for (variant in c('original', 'alpha', 'delta')) {
                        plot_heatmap(res, production, npi, variant, scale, rate)            
                    }
                }
            }
        }
    }

In [ ]:
    
    for (production in c('small', 'large')) {
            for (scale in c('All', 'Unvaccinated')) {
                for (rate in c('workplace', 'total', 'symptomatic')) {
                    
                        plot_heatmap(res, production, 1.2, 'original', scale, rate)            
            }
        }
    }